# Tabula Muris Data: From Raw Data to Analysis

In this notebook, we will go over how to start with the Chan-Zuckerberg's Tabula Muris data from start to finish.

## 1. Downloading Data

The data is public and it can be freely accessed through AWS or the CZ github repo. To download, here we provide two options:

### Option 1 Direct Download from the Chan-Zuckerberg Repo:

Here we will `wget` the data from CZ rep:

In [4]:
!wget https://github.com/chanzuckerberg/scRNA-python-workshop/raw/master/content/data.zip

--2020-10-21 18:26:26--  https://github.com/chanzuckerberg/scRNA-python-workshop/raw/master/content/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chanzuckerberg/scRNA-python-workshop/master/content/data.zip [following]
--2020-10-21 18:26:26--  https://raw.githubusercontent.com/chanzuckerberg/scRNA-python-workshop/master/content/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74104660 (71M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  70.67M   226MB/s    in 0.3s    

2020-10-21 18:26:27 (226 MB/s) - ‘data.zip’ saved [74104660/74104660]



In [5]:
!unzip data.zip

Archive:  data.zip
  inflating: brain_metadata.csv      
  inflating: glioblastoma_normalized.h5ad  
  inflating: glioblastoma_raw.h5ad   
  inflating: brain_counts.csv        
  inflating: pbmc3k.h5ad             


Note: From here on, we will be storing the data in a folder called `TM_Data` 

### Option 2: Download using AWS (recommended)

To make things much nicer, we will first install AWS CLI (command line interface) and then download the data. 

N.b: Having the AWS CLI installed on your machine/cluster will come in very handy!

In [ ]:
## get the CLI Zip
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

Now unzip the file:

In [7]:
# Now unzip and install CLI

### I HAVE AWS INSTALLED SO I GET THE FOLLOWING MESSAGE, BUT YOU SHOULD EXCEPT FULL INSTALLATION
!unzip awscliv2.zip
!sudo ./aws/install

Found preexisting AWS CLI installation: /usr/local/aws-cli/v2/current. Please rerun install script with --update flag.


Now if you don't have AWS credentials that's fine, we can do a an unauthenticated request (with no sign-in flag) since the data is available publicly

In [9]:
# list the files in the directory
!aws s3 ls s3://czb-tabula-muris/ --no-sign-request

                           PRE 10x_bam_files/
                           PRE facs_bam_files/
2018-10-08 18:38:40  147226550 TM_droplet_mat.csv.gz
2018-10-08 18:38:45  247481595 TM_droplet_mat.h5ad
2018-10-08 18:38:52  245224669 TM_droplet_mat.rds
2018-10-08 18:39:00    6120187 TM_droplet_metadata.csv
2018-10-08 18:39:00  318146414 TM_facs_mat.csv.gz
2018-10-08 18:39:11  487706533 TM_facs_mat.h5ad
2018-10-08 18:39:11  520953990 TM_facs_mat.rds
2018-10-08 18:39:25    5691123 TM_facs_metadata.csv


In [6]:
# here we want to download the files that we have access to, so for example let us download this CSV file

!aws s3 cp s3://czbiohub-tabula-muris/TM_facs_metadata.csv ./TM_Data --no-sign-request

download: s3://czbiohub-tabula-muris/TM_facs_metadata.csv to TabulaMurisData/TM_facs_metadata.csv


## Read in the Data:

Now you can use your favorite way of reading in data to python. In this NB, we will be using Pandas due to the convenience and popularity

In [8]:
import pandas as pd

brain_count = pd.read_csv("./TM_Data/brain_counts.csv", sep = ',', header = "infer", index_col = 0, error_bad_lines = True)

So at this point, we have read in the data which consists of ***unique cells*** (in rows) and ***genes*** (in columns). Let's find how many cells and how many genes we have in the data:

In [9]:
print(f"Number of Unique Cells: {len(brain_count)}")
print(f"Number of Genes: {len(brain_count.columns)}")

Number of Unique Cells: 3401
Number of Genes: 23433


### Getting Information from MetaData

Since we have a meta data associated with the brain cells, let us use it to see what information we can exctract from them

In [12]:
brain_meta = pd.read_csv("./TM_Data/brain_metadata.csv", sep = ',', header = "infer", index_col = 0, error_bad_lines = True)

In [13]:
# print the headers
brain_meta.head(1)

,cell_ontology_class,subtissue,mouse.sex,mouse.id,plate.barcode
cell,,,,,
A1.B003290.3_38_F.1.1,astrocyte,Striatum,F,3_38_F,B003290


We can extract a lot of information from the meta data, for example, let us count the number of subtissues samples in the data, and count the sex of the mice

In [15]:
print(pd.value_counts(brain_meta['subtissue']))
print("----------")
print(pd.value_counts(brain_meta['mouse.sex']))

Cortex         1149
Hippocampus     976
Striatum        723
Cerebellum      553
Name: subtissue, dtype: int64
----------
M    2694
F     707
Name: mouse.sex, dtype: int64
